In [2]:
from auto_encoder import AutoEncoder
from masked_dataset import CIFAR10DataModule
import torch
from torch import nn
from torch.nn import functional as F
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cifar10 = CIFAR10DataModule()
cifar10.setup("nothing")
cifar10_train = cifar10.train_dataloader()

in_channel_size = 3
num_epochs = 10
model = AutoEncoder(in_channel_size=3)
model.eval()

checkpoint = torch.load("last.ckpt", map_location=torch.device("cpu"))
model.load_state_dict(checkpoint["state_dict"])


Found cached dataset cifar10 (/Users/stefanbroecker/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)
100%|██████████| 2/2 [00:00<00:00, 181.39it/s]


<All keys matched successfully>

In [42]:
original_size = 224*224
batch_size = 32

mseloss = nn.MSELoss(reduction="none")
weights = torch.zeros([original_size, 2, original_size])
iterator = torch.ones(original_size)

In [46]:
for batch_idx, (masked, full, indices) in enumerate(cifar10_train):
    # masked = masked.to("cpu")
    # pass the sampled data to the model
    prediction = model(masked)
    # go through each image in the current batch
    for i in range(batch_size):
        # get loss per pixel across all channels
        loss = mseloss(prediction[i], full[i])
        single_loss = torch.sum(loss, dim=0)
        # get the indices that were in the sample
        idxs = indices[i]
        # add the losses to the index's loss tracker
        # iterate the number of samples seen
        weights[idxs, 0] += single_loss
        weights[idxs, 1] += iterator

# total loss / number of times seen
average_weight = weights[:, 0, :]/weights[:, 1, :]

# normalize across rows
normalized_weights = F.normalize(average_weight, dim=0)

torch.save(normalized_weights, "embedding_tensor.pth")

In [53]:
weights[:, 0, :].unique()

tensor([0.0000e+00, 9.6524e-06, 5.3889e-05,  ..., 1.6864e+01, 1.6927e+01,
        1.7012e+01], grad_fn=<Unique2Backward0>)

torch.Size([50176])